In [1]:
# encoding: utf-8


import os
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.decomposition import PCA
from sklearn import metrics
import tensorflow as tf

c:\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
LABELS_TXT = 'digits4000_digits_labels.txt'
VECS_TXT = 'digits4000_digits_vec.txt'
VEC_LENGTH = 784
CLASS_NUM = 10
BATCH_SIZE = 64

In [3]:
labels_df = pd.read_csv(LABELS_TXT, header=None, names=['label'])
labels = labels_df['label']
vecs_df = pd.read_csv(VECS_TXT, sep='\t', header=None, names=['f_{}'.format(i) for i in range(VEC_LENGTH)])
vecs_df = vecs_df / 255.

# check
print('labels_df.shape:', labels_df.shape)
print('labels.shape:', labels.shape)
print('vecs_df.shape:', vecs_df.shape)

labels_df.shape: (4000, 1)
labels.shape: (4000,)
vecs_df.shape: (4000, 784)


In [10]:
def to_one_hot(y_, class_num):
    label_num = y_.shape[0]
    one_hot_vecs = np.zeros((label_num, class_num))
    for i in range(label_num):
        one_hot_vecs[i][y_[i]] = 1
    return one_hot_vecs

In [13]:
X_train = vecs_df[0: 2000]
y_train = labels_df['label'][0: 2000]
X_test = vecs_df[2000: ]
y_test = labels_df['label'][2000: ]

# 乱序
X_train, y_train = shuffle(X_train, y_train)

# 重新索引
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

# check
# print('y_train:', y_train)

# 转 one-hot 向量
y_train = to_one_hot(y_train, CLASS_NUM)
y_test = to_one_hot(y_test, CLASS_NUM)

# check
# for x in y_train:
#     print(x)
    
# for x in y_test:
#     print(x)

# check
print('X_train type:', type(X_train))
print('X_test type:', type(X_test))
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_train type:', type(y_train))
print('y_test type:', type(y_test))
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

X_train type: <class 'pandas.core.frame.DataFrame'>
X_test type: <class 'pandas.core.frame.DataFrame'>
X_train shape: (2000, 784)
X_test shape: (2000, 784)
y_train type: <class 'numpy.ndarray'>
y_test type: <class 'numpy.ndarray'>
y_train shape: (2000, 10)
y_test shape: (2000, 10)


In [14]:
start_idx = 0
def next_batch(batch_size=BATCH_SIZE):
    global start_idx
    idx = start_idx
    if idx + batch_size > X_train.shape[0]:
        start_idx = 0
        return X_train[idx: ], y_train[idx: ]
    else:
        start_idx += batch_size
        return X_train[idx: idx + batch_size], y_train[idx: idx + batch_size]

In [22]:
sess = tf.InteractiveSession()

x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.truncated_normal([784, 500], stddev=0.1))
b1 = tf.Variable(tf.zeros([500]))
W2 = tf.Variable(tf.zeros([500, 10]))
b2 = tf.Variable(tf.zeros([10]))

hidden = tf.nn.relu(tf.matmul(x, W1) + b1)
hidden_drop = tf.nn.dropout(hidden, keep_prob)
y = tf.nn.softmax(tf.matmul(hidden_drop, W2) + b2)

# 计算交叉熵
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

train_step = tf.train.AdagradOptimizer(0.3).minimize(cross_entropy)
tf.global_variables_initializer().run()

for i in range(0, 5000):
    batch_xs, batch_ys = next_batch(64)
    train_step.run({x: batch_xs, y_: batch_ys, keep_prob: 0.75})
    if i % 100 == 0 and i != 0:
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        print('step={}, test accuracy={:4f}'.format(i, accuracy.eval({x: X_test, y_: y_test, keep_prob: 1.0})))
        
sess.close()

step=100, test accuracy=0.917500
step=200, test accuracy=0.924500
step=300, test accuracy=0.935500
step=400, test accuracy=0.933000
step=500, test accuracy=0.938000
step=600, test accuracy=0.935500
step=700, test accuracy=0.933500
step=800, test accuracy=0.939500
step=900, test accuracy=0.936000
step=1000, test accuracy=0.938000
step=1100, test accuracy=0.939500
step=1200, test accuracy=0.937500
step=1300, test accuracy=0.938000
step=1400, test accuracy=0.940000
step=1500, test accuracy=0.933000
step=1600, test accuracy=0.939000
step=1700, test accuracy=0.938500
step=1800, test accuracy=0.938500
step=1900, test accuracy=0.935500
step=2000, test accuracy=0.937000
step=2100, test accuracy=0.936000
step=2200, test accuracy=0.937500
step=2300, test accuracy=0.938000
step=2400, test accuracy=0.937500
step=2500, test accuracy=0.939000
step=2600, test accuracy=0.938500
step=2700, test accuracy=0.938500
step=2800, test accuracy=0.940000
step=2900, test accuracy=0.938500
step=3000, test accurac